## 141 期間内におけるmonth, weekのカバー率

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [17]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([df_train, df_test], axis=0)
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')
df_trans = pd.concat([auth1, df_new], axis=0)

In [3]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [5]:
for df in [df_hist, df_new]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
#     df['hour'] = df['purchase_date'].dt.hour
#     df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
#     df['date_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)

In [6]:
df_hist.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)
df_hist['yyyymm'] =  df_hist['purchase_date'].map(lambda x: str(x)[:7] + '-01')
df_hist['yyyy_week'] = df_hist['yyyymm'].map(lambda x: str(x)[:4]) + df_hist['weekofyear'].map(lambda x: '-0' + str(x) if len(str(x))==1 else '-' + str(x) )

df_new.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)
df_new['yyyymm'] =  df_hist['purchase_date'].map(lambda x: str(x)[:7] + '-01')
df_new['yyyy_week'] = df_hist['yyyymm'].map(lambda x: str(x)[:4]) + df_hist['weekofyear'].map(lambda x: '-0' + str(x) if len(str(x))==1 else '-' + str(x) )
df_hist.head()

,authorized_flag,card_id,month_lag,purchase_date,state_id,subsector_id,weekofyear,yyyymm,yyyy_week
0,Y,C_ID_4e6213e9bc,-8,2017-06-25 15:33:07,16,37,25,2017-06-01,2017-25
1,Y,C_ID_4e6213e9bc,-7,2017-07-15 12:10:45,16,16,28,2017-07-01,2017-28
2,Y,C_ID_4e6213e9bc,-6,2017-08-09 22:04:29,16,37,32,2017-08-01,2017-32
3,Y,C_ID_4e6213e9bc,-5,2017-09-02 10:06:26,16,34,35,2017-09-01,2017-35
4,Y,C_ID_4e6213e9bc,-11,2017-03-10 01:14:19,16,37,10,2017-03-01,2017-10


In [7]:
auth1 = df_hist[df_hist['authorized_flag']=='Y']
auth0 = df_hist[df_hist['authorized_flag']=='N']

### 連続week, month利用回数、期間内week, monthカバー率を特徴にする

In [28]:
# auth1
df = auth1

df_month = df[[key, 'yyyymm']].drop_duplicates()
df_month_max = df_month.groupby(key)['yyyymm'].max().reset_index().rename(columns={'yyyymm':'yyyymm_max'})
df_month_min = df_month.groupby(key)['yyyymm'].min().reset_index().rename(columns={'yyyymm':'yyyymm_min'})
df_month_cnt = df_month.groupby(key)['yyyymm'].count().reset_index().rename(columns={'yyyymm':'yyyymm_cnt'})

df_term = df_month_max.merge(df_month_min, how='inner', on=key).merge(df_month_cnt, how='inner', on=key)
df_term['term'] = (pd.to_datetime(df_term['yyyymm_max']) - pd.to_datetime(df_term['yyyymm_min'])).dt.days//30
df_term['yyyymm_ratio'] = (df_term['yyyymm_cnt']-1) / (df_term['term'])
display(df_term.head(20))

df = auth0

df_month = df[[key, 'yyyymm']].drop_duplicates()
df_month_max = df_month.groupby(key)['yyyymm'].max().reset_index().rename(columns={'yyyymm':'yyyymm_max'})
df_month_min = df_month.groupby(key)['yyyymm'].min().reset_index().rename(columns={'yyyymm':'yyyymm_min'})
df_month_cnt = df_month.groupby(key)['yyyymm'].count().reset_index().rename(columns={'yyyymm':'yyyymm_cnt'})

df_term_0 = df_month_max.merge(df_month_min, how='inner', on=key).merge(df_month_cnt, how='inner', on=key)
df_term_0['term'] = (pd.to_datetime(df_term_0['yyyymm_max']) - pd.to_datetime(df_term_0['yyyymm_min'])).dt.days//30
df_term_0['yyyymm_ratio'] = (df_term_0['yyyymm_cnt']-1) / df_term_0['term']
df_term_0.head(20)

df = df_trans

df_month = df[[key, 'yyyymm']].drop_duplicates()
df_month_max = df_month.groupby(key)['yyyymm'].max().reset_index().rename(columns={'yyyymm':'yyyymm_max'})
df_month_min = df_month.groupby(key)['yyyymm'].min().reset_index().rename(columns={'yyyymm':'yyyymm_min'})
df_month_cnt = df_month.groupby(key)['yyyymm'].count().reset_index().rename(columns={'yyyymm':'yyyymm_cnt'})

df_term_all = df_month_max.merge(df_month_min, how='inner', on=key).merge(df_month_cnt, how='inner', on=key)
df_term_all['term'] = (pd.to_datetime(df_term_all['yyyymm_max']) - pd.to_datetime(df_term_all['yyyymm_min'])).dt.days//30
df_term_all['yyyymm_ratio'] = (df_term_all['yyyymm_cnt']-1) / df_term_all['term']
df_term_all.head(20)

,card_id,yyyymm_max,yyyymm_min,yyyymm_cnt,term,yyyymm_ratio
0,C_ID_00007093c1,2018-02-01,2017-02-01,13,12,1.000000
1,C_ID_0001238066,2018-02-01,2017-09-01,6,5,1.000000
2,C_ID_0001506ef0,2018-02-01,2017-01-01,13,13,0.923077
3,C_ID_0001793786,2017-10-01,2017-01-01,10,9,1.000000
4,C_ID_000183fdda,2018-02-01,2017-09-01,6,5,1.000000
5,C_ID_00024e244b,2018-02-01,2017-01-01,13,13,0.923077
6,C_ID_0002709b5a,2018-02-01,2017-04-01,11,10,1.000000
7,C_ID_00027503e2,2018-01-01,2017-05-01,9,8,1.000000
8,C_ID_000298032a,2018-01-01,2017-02-01,8,11,0.636364
9,C_ID_0002ba3c2e,2017-09-01,2017-01-01,6,8,0.625000


,card_id,yyyymm_max,yyyymm_min,yyyymm_cnt,term,yyyymm_ratio
0,C_ID_00007093c1,2018-02-01,2017-02-01,13,12,1.000000
1,C_ID_0001238066,2018-02-01,2017-08-01,7,6,1.000000
2,C_ID_0001506ef0,2018-02-01,2017-01-01,13,13,0.923077
3,C_ID_0001793786,2018-02-01,2017-01-01,14,13,1.000000
4,C_ID_000183fdda,2018-02-01,2017-07-01,8,7,1.000000
5,C_ID_00024e244b,2018-02-01,2017-01-01,13,13,0.923077
6,C_ID_0002709b5a,2018-02-01,2017-04-01,11,10,1.000000
7,C_ID_00027503e2,2018-01-01,2017-05-01,9,8,1.000000
8,C_ID_000298032a,2018-01-01,2017-02-01,9,11,0.727273
9,C_ID_0002ba3c2e,2018-01-01,2017-01-01,8,12,0.583333


In [29]:
df_term.rename(columns={'yyyymm_ratio':'auth1_yyyymm_ratio'}, inplace=True)
df_term_0.rename(columns={'yyyymm_ratio':'auth0_yyyymm_ratio'}, inplace=True)
df_term = df_term[[key, 'auth1_yyyymm_ratio']].merge(df_term_0[[key, 'auth0_yyyymm_ratio']], how='left', on=key)

df_term_all.rename(columns={'yyyymm_ratio':'all_yyyymm_ratio'}, inplace=True)
df_month_cover = df_term.merge(df_term_all[[key, 'all_yyyymm_ratio']], how='left', on=key)

print(df_month_cover.shape)
df_month_cover.head()

(325540, 4)


,card_id,auth1_yyyymm_ratio,auth0_yyyymm_ratio,all_yyyymm_ratio
0,C_ID_00007093c1,1.000000,1.000000,1.000000
1,C_ID_0001238066,1.000000,0.500000,1.000000
2,C_ID_0001506ef0,0.923077,0.250000,0.923077
3,C_ID_0001793786,1.000000,0.833333,1.000000
4,C_ID_000183fdda,1.000000,0.500000,1.000000


### 週

In [23]:
df = auth1

week_val = df_week['yyyy_week'].drop_duplicates().sort_values().to_frame()
week_val['index'] = np.arange(len(week_val)) + 1

df_week = df[[key, 'yyyy_week']].drop_duplicates()
df_week = df_week.merge(week_val, how='inner', on='yyyy_week')

df_week_max = df_week.groupby(key)['index'].max().reset_index().rename(columns={'index':'yyyy_week_max'})
df_week_min = df_week.groupby(key)['index'].min().reset_index().rename(columns={'index':'yyyy_week_min'})
df_week_cnt = df_week.groupby(key)['yyyy_week'].count().reset_index().rename(columns={'yyyy_week':'yyyy_week_cnt'})

df_term = df_week_max.merge(df_week_min, how='inner', on=key).merge(df_week_cnt, how='inner', on=key)
df_term['term'] = df_term['yyyy_week_max'] - df_term['yyyy_week_min']
df_term['yyyy_week_ratio'] = (df_term['yyyy_week_cnt']-1) / (df_term['term'])
display(df_term.head(20))

df = auth0

df_week = df[[key, 'yyyy_week']].drop_duplicates()
df_week = df_week.merge(week_val, how='inner', on='yyyy_week')

df_week_max = df_week.groupby(key)['index'].max().reset_index().rename(columns={'index':'yyyy_week_max'})
df_week_min = df_week.groupby(key)['index'].min().reset_index().rename(columns={'index':'yyyy_week_min'})
df_week_cnt = df_week.groupby(key)['yyyy_week'].count().reset_index().rename(columns={'yyyy_week':'yyyy_week_cnt'})

df_term_0 = df_week_max.merge(df_week_min, how='inner', on=key).merge(df_week_cnt, how='inner', on=key)
df_term_0['term'] = df_term_0['yyyy_week_max'] - df_term_0['yyyy_week_min']
df_term_0['yyyy_week_ratio'] = (df_term_0['yyyy_week_cnt']-1) / df_term_0['term']
display(df_term_0.head(20))

df = df_trans

df_week = df[[key, 'yyyy_week']].drop_duplicates()
df_week = df_week.merge(week_val, how='inner', on='yyyy_week')

df_week_max = df_week.groupby(key)['index'].max().reset_index().rename(columns={'index':'yyyy_week_max'})
df_week_min = df_week.groupby(key)['index'].min().reset_index().rename(columns={'index':'yyyy_week_min'})
df_week_cnt = df_week.groupby(key)['yyyy_week'].count().reset_index().rename(columns={'yyyy_week':'yyyy_week_cnt'})

df_term_all = df_week_max.merge(df_week_min, how='inner', on=key).merge(df_week_cnt, how='inner', on=key)
df_term_all['term'] = df_term_all['yyyy_week_max'] - df_term_all['yyyy_week_min']
df_term_all['yyyy_week_ratio'] = (df_term_all['yyyy_week_cnt']-1) / df_term_all['term']
df_term_all.head(20)

,card_id,yyyy_week_max,yyyy_week_min,yyyy_week_cnt,term,yyyy_week_ratio
0,C_ID_00007093c1,58,10,19,48,0.375000
1,C_ID_0001238066,58,52,2,6,0.166667
2,C_ID_0001506ef0,59,23,4,36,0.083333
3,C_ID_0001793786,37,9,13,28,0.428571
4,C_ID_000183fdda,43,32,4,11,0.272727
5,C_ID_00024e244b,58,14,7,44,0.136364
6,C_ID_0002709b5a,55,2,5,53,0.075472
7,C_ID_00027503e2,55,23,9,32,0.250000
8,C_ID_000298032a,38,15,2,23,0.043478
9,C_ID_0002ba3c2e,43,3,7,40,0.150000


,card_id,yyyy_week_max,yyyy_week_min,yyyy_week_cnt,term,yyyy_week_ratio
0,C_ID_00007093c1,61,7,36,54,0.648148
1,C_ID_0001238066,61,33,25,28,0.857143
2,C_ID_0001506ef0,59,2,29,57,0.491228
3,C_ID_0001793786,59,3,44,56,0.767857
4,C_ID_000183fdda,60,28,27,32,0.812500
5,C_ID_00024e244b,58,1,21,57,0.350877
6,C_ID_0002709b5a,60,17,24,43,0.534884
7,C_ID_00027503e2,55,21,18,34,0.500000
8,C_ID_000298032a,56,7,14,49,0.265306
9,C_ID_0002ba3c2e,54,1,17,53,0.301887


In [27]:
df_term.rename(columns={'yyyy_week_ratio':'auth1_yyyy_week_ratio'}, inplace=True)
df_term_0.rename(columns={'yyyy_week_ratio':'auth0_yyyy_week_ratio'}, inplace=True)
df_term_all.rename(columns={'yyyy_week_ratio':'all_yyyy_week_ratio'}, inplace=True)
df_week_cover = df_term_all[[key, 'all_yyyy_week_ratio']].merge(df_term_0[[key, 'auth0_yyyy_week_ratio']], how='left', on=key).merge(df_term[[key, 'auth1_yyyy_week_ratio']], how='left', on=key)

print(df_week_cover.shape)
df_week_cover.head()

(325540, 4)


,card_id,all_yyyy_week_ratio,auth0_yyyy_week_ratio,auth1_yyyy_week_ratio
0,C_ID_00007093c1,0.648148,0.375000,0.629630
1,C_ID_0001238066,0.857143,0.166667,1.000000
2,C_ID_0001506ef0,0.491228,0.083333,0.473684
3,C_ID_0001793786,0.767857,0.428571,0.756098
4,C_ID_000183fdda,0.812500,0.272727,0.960000


In [30]:
for df in [df_month_cover, df_week_cover]:
    df_train = df_train.merge(df, how='left', on=key)
    df_test = df_test.merge(df, how='left', on=key)
    
print(df_train.shape)
df_train.head()

(201917, 12)


,first_active_month,card_id,feature_1,feature_2,feature_3,target,auth1_yyyymm_ratio,auth0_yyyymm_ratio,all_yyyymm_ratio,all_yyyy_week_ratio,auth0_yyyy_week_ratio,auth1_yyyy_week_ratio
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,1.000000,1.000000,1.000000,0.683333,0.333333,1.000000
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,1.000000,0.555556,1.000000,0.946429,0.186047,0.946429
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,0.846154,1.000000,0.846154,0.406780,0.125000,0.389831
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,1.000000,NaN,0.846154,0.423729,NaN,0.863636
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,1.000000,1.000000,1.000000,0.672414,0.166667,1.000000


In [32]:
fname = '141_cov'
ignore_features = ['first_active_month', 'card_id']

for col in df_train.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('yyyy')):continue
#     if not(col.count('trans')):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)